In [1]:
import os # импортируем библиотеку для работы с файловой системой

In [2]:
%pwd # проверяем текущую директорию

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../") # переходим в корневую директорию проекта

In [4]:
%pwd # проверяем текущую директорию

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project'

Update entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

Update the configuration manager in src config

In [6]:
from textSummarizer.constants import * # импортируем константы
from textSummarizer.utils.common import read_yaml, create_directories # импортируем функции

In [7]:
class ConfigurationManager: # создаем класс конфигурации
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig: 
        # создаем функцию для получения конфигурации модели
        config = self.config.model_trainer # получаем конфигурацию модели
        params = self.params.TrainingArguments # получаем параметры обучения

        create_directories([config.root_dir]) # создаем директорию для сохранения модели

        model_trainer_config = ModelTrainerConfig( # создаем конфигурацию модели
            root_dir=config.root_dir, # указываем директорию для сохранения модели
            data_path=config.data_path, # указываем путь к данным
            model_ckpt = config.model_ckpt, # указываем путь к чекпоинту модели
            num_train_epochs = params.num_train_epochs, # указываем количество эпох обучения
            warmup_steps = params.warmup_steps, # указываем количество шагов для прогрева
            per_device_train_batch_size = params.per_device_train_batch_size, # указываем размер батча
            weight_decay = params.weight_decay, # указываем коэффициент веса
            logging_steps = params.logging_steps, # указываем шаги для логирования
            evaluation_strategy = params.evaluation_strategy, # указываем стратегию оценки
            eval_steps = params.evaluation_strategy, # указываем шаги для оценки
            save_steps = params.save_steps, # указываем шаги для сохранения
            gradient_accumulation_steps = params.gradient_accumulation_steps 
            # указываем шаги для накопления градиента
        )

        return model_trainer_config # возвращаем конфигурацию модели

Update the conponents


In [8]:
from transformers import TrainingArguments, Trainer # импортируем классы для обучения модели
from transformers import DataCollatorForSeq2Seq # импортируем класс для создания коллатора данных
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer # импортируем классы для работы с моделью
from datasets import load_dataset, load_from_disk # импортируем классы для работы с датасетом
import torch # импортируем библиотеку для работы с тензорами

c:\Users\Mikhnin\miniconda3\envs\project_1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer: # создаем класс для обучения модели
    def __init__(self, config: ModelTrainerConfig): # создаем конструктор класса
        self.config = config # инициализируем конфигурацию модели


    
    def train(self): # создаем функцию для обучения модели
        device = "cuda" if torch.cuda.is_available() else "cpu" # получаем доступное устройство для обучения
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt) # загружаем токенайзер
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device) # загружаем модель
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus) # создаем коллатор данных
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path) # загружаем датасет

      
        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,  
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"]) # создаем тренера
        
        trainer.train() # запускаем обучение модели

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model")) # сохраняем модель
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer")) # сохраняем токенайзер

Update the pipeline

In [ ]:
try: # создаем блок try-except
    config = ConfigurationManager() # создаем конфигурацию
    model_trainer_config = config.get_model_trainer_config() # получаем конфигурацию модели
    model_trainer_config = ModelTrainer(config=model_trainer_config) # создаем конфигурацию для обучения модели
    model_trainer_config.train() # обучаем модель
except Exception as e: # создаем блок для обработки ошибок
    raise e # выводим ошибку